Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [2]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

NameError: name 'train_dataset' is not defined

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Logistic Network
batch_size = 128
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularization_term = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = loss + beta*regularization_term
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.867817
Minibatch accuracy: 13.3%
Validation accuracy: 14.0%
Minibatch loss at step 500: 1.007730
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.809247
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 0.879163
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 0.670312
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 0.751855
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.953568
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Test accuracy: 88.2%


In [8]:
# Neural net 
batch_size = 128
hidden_size = 1024


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size,hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  # Training computation.
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)
  
  output_weights =  tf.Variable(
    tf.truncated_normal([hidden_size,num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  

  logits = tf.matmul(hidden_output,output_weights) + output_biases
  loss =  tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  
  regularization_term = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = loss + beta*regularization_term
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden,output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)
     

        

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3461.028076
Minibatch accuracy: 10.9%
Validation accuracy: 24.0%
Minibatch loss at step 500: 21.333048
Minibatch accuracy: 81.2%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.896691
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.793941
Minibatch accuracy: 79.7%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.640093
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.684053
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.848595
Minibatch accuracy: 80.5%
Validation accuracy: 83.7%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# Neural net 
batch_size = 20
hidden_size = 1024


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size,hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  # Training computation.
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)
  
  output_weights =  tf.Variable(
    tf.truncated_normal([hidden_size,num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  

  logits = tf.matmul(hidden_output,output_weights) + output_biases
  loss =  tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  
  regularization_term = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = loss + beta*regularization_term
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden,output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)
     

        

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3449.357422
Minibatch accuracy: 20.0%
Validation accuracy: 18.9%
Minibatch loss at step 500: 25.181969
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.833117
Minibatch accuracy: 65.0%
Validation accuracy: 73.2%
Minibatch loss at step 1500: 0.660871
Minibatch accuracy: 90.0%
Validation accuracy: 76.3%
Minibatch loss at step 2000: 1.056839
Minibatch accuracy: 70.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.751415
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.747152
Minibatch accuracy: 90.0%
Validation accuracy: 76.8%
Test accuracy: 83.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
# Neural net 
batch_size = 128
hidden_size = 1024


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size,hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  # Training computation.
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights) + hidden_biases)
  
  output_weights =  tf.Variable(
    tf.truncated_normal([hidden_size,num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  

  logits = tf.matmul(tf.nn.dropout(hidden_output,0.5) ,output_weights) + output_biases
  loss =  tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  
  regularization_term = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = loss + beta*regularization_term
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden,output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)
     

        

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3569.027100
Minibatch accuracy: 11.7%
Validation accuracy: 30.5%
Minibatch loss at step 500: 21.539318
Minibatch accuracy: 82.0%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.945435
Minibatch accuracy: 83.6%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 0.862821
Minibatch accuracy: 80.5%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.695712
Minibatch accuracy: 84.4%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 0.731412
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.916746
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Test accuracy: 89.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [20]:
# Neural net 
batch_size = 128
hidden_size = 2048
beta = 5e-4
import math

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size,hidden_size],stddev = 1/math.sqrt(float(image_size*image_size))))
  hidden_biases_1 = tf.Variable(tf.zeros([hidden_size]))
  
  # Training computation for 1st hidden layer
  hidden_output_1 = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights_1) + hidden_biases_1)
  
  hidden_weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size,hidden_size],stddev = 1/math.sqrt(float(image_size*image_size))))
  hidden_biases_2 = tf.Variable(tf.zeros([hidden_size]))
    
  # Training computation for 2nd hidden layer
  hidden_output_2 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_output_1,0.5),hidden_weights_2) + hidden_biases_2)
    
  # Training computation for final layer  
  output_weights =  tf.Variable(
    tf.truncated_normal([hidden_size,num_labels],stddev = 1/math.sqrt(float(image_size*image_size))))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  

  logits = tf.matmul(tf.nn.dropout(hidden_output_2,0.5),output_weights) + output_biases
  loss =  tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  
  regularization_term = tf.nn.l2_loss(hidden_weights_1) + tf.nn.l2_loss(hidden_biases_1) + tf.nn.l2_loss(hidden_weights_2) + tf.nn.l2_loss(hidden_biases_2) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = loss + beta*regularization_term
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step,100000,0.96,staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights_1) + hidden_biases_1)
  valid_hidden_2 = tf.nn.relu(tf.matmul(valid_hidden_1, hidden_weights_2) + hidden_biases_2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_2,output_weights) + output_biases)
  test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights_1) + hidden_biases_1)
  test_hidden_2 = tf.nn.relu(tf.matmul(test_hidden_1, hidden_weights_2) + hidden_biases_2)  
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_2, output_weights) + output_biases)
     

        

num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 4.169432
Minibatch accuracy: 8.6%
Validation accuracy: 30.2%
Minibatch loss at step 500: 1.836454
Minibatch accuracy: 78.1%
Validation accuracy: 85.2%
Minibatch loss at step 1000: 1.421570
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 1500: 1.369677
Minibatch accuracy: 82.8%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 1.065762
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 2500: 0.894617
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Minibatch loss at step 3000: 0.995496
Minibatch accuracy: 84.4%
Validation accuracy: 88.0%
Minibatch loss at step 3500: 0.813931
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 4000: 0.798304
Minibatch accuracy: 85.2%
Validation accuracy: 88.4%
Minibatch loss at step 4500: 0.651540
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.629926
Minibatch accuracy: 90.6%
Validation accuracy